In [1]:
import logging

import numpy as np
import tensorflow as tf
print(tf.__version__)
from tqdm.notebook import tqdm

import sys
sys.path.append("agent/util")
from simulation import EpisodeRunner,logger as simulation_logger
simulation_logger.setLevel(logging.ERROR)

from dnn_model import DataSetCreationActionSolver
from strategy import Strategy,RandomMixStrategy,RandomStrategy

sys.path.append("agent")
from CBEngine_round3 import CBEngine_round3 as CBEngine_rllib_class

2.5.0


/usr/local/lib/python3.7/dist-packages/ray/autoscaler/_private/cli_logger.py:61: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  "update your install command.", FutureWarning)


In [2]:
#####################################
###parameters for simulator
simulator_cfg_file = "cfg/simulator_round2.cfg"
metric_period = 20
earlyStoppingDelayIndex=1.6
bufferLength=10000
DEBUG=False

###parameters for DNN agent
dnnCheckpointFileName="round3.ckpt"
#dnnNumSegmentInbound=18
#dnnNumSegmentOutbound=18
#dimVehiclesVec=64
#unitsDict={
#    'inbound':[64,64,64,64],
#}

###parameters for rule base agent
maxDepth=4
timeThresForCalcReward=10.2
prohibitDecreasingGoalDistance=True
prohibitDecreasingSpeed=True

###parameters for mixing strategy 
dnnStrategyRatio=0.5
ruleStrategyRatio=0.5

###parameters for actionSolver
tfrecordPath="tfrecord/round3.tfrecord"
saveNumSegmentInbound = 18
saveNumSegmentOutbound = 18

#####################################
#strategy=RandomMixStrategy({
#    Strategy.createStrategy(
#        'run_distance',
#        {
#            'maxDepth':maxDepth,
#            'timeThresForCalcReward':timeThresForCalcReward,
#            'prohibitDecreasingGoalDistance':prohibitDecreasingGoalDistance,
#            'prohibitDecreasingSpeed':prohibitDecreasingSpeed,
#        }
#    ):ruleStrategyRatio,
#    Strategy.createStrategy(
#        'dnn_run_distance',{
#            'checkpointPath':"ckpt/"+dnnCheckpointFileName,
#            'numSegmentInbound':dnnNumSegmentInbound,
#            'numSegmentOutbound':dnnNumSegmentOutbound,
#            'unitsDict':unitsDict,
#            'dimVehiclesVec':dimVehiclesVec,
#        }
#    ):dnnStrategyRatio,
#})

strategy = RandomStrategy()

dataSetCreator=DataSetCreationActionSolver(
    strategy=strategy
)
runner=EpisodeRunner(
    dataSetCreator,
    CBEngine_rllib_class,
    simulator_cfg_file,
    metric_period,
    debug=DEBUG
)
            
runner.runLoop(
    "strategy",
    earlyStoppingDelayIndex,
    breakReplayBufferLength=bufferLength,
    tqdm=tqdm
)

dataSetCreator.createTFRecord(tfrecordPath)

0it [00:00, ?it/s]

False
before action
after action
end action
True
before action
after action
end action
True
before action
after action
end action
True
before action
after action
end action
True
before action
after action
end action
True
before action
after action
end action
True


AssertionError: +2 hop distanace

In [ ]:
dataSetCreator.sectionNum